# After the cleaning in cleaning file i will join the files togither

0   &emsp; 5059 <br>
2    &emsp; 394<br>
4    &emsp; 350<br>
1    &emsp; 282<br>
5   &emsp;  261<br>
3  &emsp;   184<br>
6   &emsp;  117<br>
7   &emsp;  115<br>
<br>
each day indicate how many apartment sold 0 means not sold <br>
1 means sold in day 7/8/2022<br>
2 means sold in day 8/8/2022 <br>
...         ...     ...<br>
7 means sold in day 12/8/2022<br>
<br>
i would like to say that i have the data in 1st , 2nd and 3rd of this month but i don't have the data in the 4th and 5th of this month so i started in the 6th 
<br>
1/8  i have <br>
2/8  i have <br>
3/8  i have <br>
4/8  i don't have <br>
5/8  i don't have <br>
6/8  i have <br>
7/8  i have <br>
8/8  i have <br>
9/8  i have <br>
10/8 i have <br>
11/8 i have <br>
12/8 i have  <br>
13/8 i have  <br>
...

In [15]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import datetime
from datetime import timedelta
import ast
import json    
import missingno as msno

In [16]:
yest =pd.read_csv('../data/aqar.csv') # the main which is 6-8-2022
Today =  pd.read_csv('../data/21-8-2022.csv') #the new day file

In [17]:
Today.columns

Index(['user_id', 'id', 'title', 'price', 'content', 'imgs', 'refresh', 'beds',
       'livings', 'wc', 'area', 'street_width', 'age', 'last_update',
       'ketchen', 'ac', 'furnished', 'location', 'path', 'district', 'width',
       'length', 'advertiser_type', 'create_time', 'review', 'profileImg',
       'UserName', 'iam_verified'],
      dtype='object')

In [18]:
yest.DayOfRent.value_counts()

0     3835
2      394
4      350
1      282
5      261
8      245
10     212
14     188
12     186
3      184
9      168
11     141
6      117
7      115
13      84
Name: DayOfRent, dtype: int64

In [19]:
day = 15 #this the day of the file from the 6th

In [20]:
# yest.sold.value_counts()

In [21]:
Today.columns

Index(['user_id', 'id', 'title', 'price', 'content', 'imgs', 'refresh', 'beds',
       'livings', 'wc', 'area', 'street_width', 'age', 'last_update',
       'ketchen', 'ac', 'furnished', 'location', 'path', 'district', 'width',
       'length', 'advertiser_type', 'create_time', 'review', 'profileImg',
       'UserName', 'iam_verified'],
      dtype='object')

In [22]:
if yest.columns[0] == 'Unnamed: 0':
    yest.drop('Unnamed: 0' , axis = 1 , inplace=True)

In [23]:
if Today.columns[0] == 'Unnamed: 0':
    Today.drop('Unnamed: 0' , axis = 1 , inplace=True)

In [24]:
# yest['sold'] = 0 #we used this for the first time only to create the column

In [25]:
yest[yest['DayOfRent'] == 0 ].shape

(3835, 31)

In [26]:
df3 = yest.merge(Today, on='id', how='left', indicator=True) #left join that will contain the data in the main file and the data in both (in both means not sold yet)(right means pulish in after 6th)
df3 = df3.loc[(df3['_merge'] == 'left_only') & (df3['DayOfRent'] == 0 )] 


In [27]:
# this will filter if it is both(inner ) so now i have only the left and for df3['sold'] == 0 this becouse if ot is 1 then this mean i checked it bedore and it is sold in 1 
df3.drop(df3.filter(regex='_y$').columns, axis=1, inplace=True) # delete the extra columns 

In [28]:
df3.shape

(82, 32)

In [29]:
df3['DayOfRent'] = day #this will fill all the data above with the day of it 

In [30]:
df3['DayOfRent']

33      15
113     15
126     15
170     15
297     15
        ..
6242    15
6286    15
6471    15
6615    15
6679    15
Name: DayOfRent, Length: 82, dtype: int64

In [31]:
for i in list(df3['id']) : 
    yest.loc[yest["id"] == i, "DayOfRent"] = day #now we fill the original data with day


In [32]:
yest['DayOfRent'].value_counts()

0     3753
2      394
4      350
1      282
5      261
8      245
10     212
14     188
12     186
3      184
9      168
11     141
6      117
7      115
13      84
15      82
Name: DayOfRent, dtype: int64

In [33]:
# yest.rename(columns = {'sold':'DayOfRent'}, inplace = True)

In [34]:
yest['DayOfRent'].value_counts()

0     3753
2      394
4      350
1      282
5      261
8      245
10     212
14     188
12     186
3      184
9      168
11     141
6      117
7      115
13      84
15      82
Name: DayOfRent, dtype: int64

In [35]:
yest.to_csv('../data/aqar.csv' , index=False)

now  for the updateDate phase 